# Ejercicio 1: Introducción a Recuperación de Información

## Objetivo de la práctica
- Entender el problema de **buscar información** en colecciones de texto.
- Comprender por qué se necesita un **índice invertido** en recuperación de información.
- Programar una primera solución manual y luego optimizarla con un índice.
- Evaluar la mejora en tiempos de búsqueda cuando usamos estructuras adecuadas.

## Parte 1: Búsqueda lineal en documentos

### Actividad
1. Se te proporcionará un dataset con reviews de películas.
2. Escribe una función que:
   - Lea todos los documentos.
   - Busque una palabra ingresada por el usuario.
   - Muestre en qué documentos aparece la palabra.

In [83]:
import pandas as pd

In [84]:
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [85]:
def buscar(docs, query):
    query = query.lower()
    mask = docs.str.lower().str.contains(query, na=False)
    resultados = docs[mask]
    return resultados


In [86]:
buscar(df['review'], 'family')

3        Basically there's a family where a little boy ...
27       This film tried to be too many things all at o...
28       This movie was so frustrating. Everything seem...
31       "Ardh Satya" is one of the finest film ever ma...
44       This movie struck home for me. Being 29, I rem...
                               ...                        
49881    Definitely an odd debut for Michael Madsen. Ma...
49892    Thin story concerns two small town brothers an...
49925    What a disaster! Normally, when one critiques ...
49927    A group of cats look to find their way home af...
49980    A stunning film of high quality.<br /><br />Ap...
Name: review, Length: 4051, dtype: object

## Parte 2: Construcción de un índice invertido

### Actividad
1. Escribe un programa que:
   - Recorra todos los documentos.
   - Construya un **índice invertido**, es decir, un diccionario donde:
     - Cada palabra clave apunta a una lista de documentos donde aparece.

2. Escribe una nueva función de búsqueda que:
   - Consulte directamente el índice para encontrar los documentos relevantes.
   - Sea mucho más rápida que la búsqueda lineal.

In [87]:
# Columna con las palabras
df['tokens'] = (
    df['review']
    .str.lower()                                   # convertir a minúsculas
    .str.findall(r'\b\w+\b')                       # extraer solo palabras
)

# Con cada palabra como una fila
df_tokens = df[['tokens']].explode('tokens').reset_index(names='doc_id')

# Eliminar duplicados
df_tokens = df_tokens.drop_duplicates(subset=['tokens', 'doc_id'])

# Agrupar por palabra y juntar los índices de los documentos
indice_invertido = (
    df_tokens.groupby('tokens')['doc_id']
    .apply(list)
    .to_dict()
)


In [88]:
len(indice_invertido['family'])

4046

In [89]:
import re
def buscar_rapido(query, indice, df):

    palabras = re.findall(r'\b\w+\b', query.lower())

    # Verificar si alguna palabra no está en el índice
    listas = [set(indice.get(p, [])) for p in palabras]

    if not listas or all(len(s) == 0 for s in listas):
        print(f"Ninguna de las palabras '{query}' aparece en el índice.")
        return

    # Intersección → documentos que contienen todas las palabras del query
    docs_resultado = set.intersection(*listas)
    print(f"'{query}' aparece en {len(docs_resultado)} documentos.\n")
    display(df.loc[list(docs_resultado), ['review', 'sentiment']].head(5))

In [90]:
buscar_rapido("family", indice_invertido, df)

'family' aparece en 4046 documentos.



,review,sentiment
40960,"Bad Actors, bad filming, choppy dialog, shallo...",negative
24577,Really enjoyed this little movie. It's a movin...,positive
40961,Strangers with candy overacts in all the wrong...,negative
3,Basically there's a family where a little boy ...,negative
8195,"Paul Hennessy and his wife, Cate must deal wit...",positive


## Parte 3: Evaluación de tiempos de búsqueda
### Actividad

1. Realiza la búsqueda de varias palabras usando:
      -  Corpus pequeño.
      -  Corpus grande.
2. Mide el tiempo de ejecución:
      -  Para búsqueda lineal.
      -  Para búsqueda usando índice invertido.
      -  Grafica o presenta los resultados en una tabla comparativa.

In [91]:
df_large= pd.read_csv('/kaggle/input/rotten-tomatoes-movies-and-critic-reviews-dataset/rotten_tomatoes_critic_reviews.csv')
df_large

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...
...,...,...,...,...,...,...,...,...
1130012,m/zulu_dawn,Chuck O'Leary,False,Fantastica Daily,Rotten,2/5,2005-11-02,NaN
1130013,m/zulu_dawn,Ken Hanke,False,"Mountain Xpress (Asheville, NC)",Fresh,3.5/5,2007-03-07,"Seen today, it's not only a startling indictme..."
1130014,m/zulu_dawn,Dennis Schwartz,False,Dennis Schwartz Movie Reviews,Fresh,B+,2010-09-16,A rousing visual spectacle that's a prequel of...
1130015,m/zulu_dawn,Christopher Lloyd,False,Sarasota Herald-Tribune,Rotten,3.5/5,2011-02-28,"A simple two-act story: Prelude to war, and th..."


In [92]:
df_large['tokens'] = (
    df_large['review_content']
    .str.lower()
    .str.findall(r'\b\w+\b')
)

df_tokens_large = df_large[['tokens']].explode('tokens').reset_index(names='doc_id')
df_tokens_large = df_tokens_large.drop_duplicates(subset=['tokens', 'doc_id'])

indice_invertido_large = (
    df_tokens_large.groupby('tokens')['doc_id']
    .apply(list)
    .to_dict()
)
print("Índice construido.\n")

Índice construido.



In [93]:
def buscar_rapido_flexible(query, indice, df):
    palabras = re.findall(r'\b\w+\b', query.lower())
    
    listas = [set(indice.get(p, [])) for p in palabras]
    
    if not listas or all(len(s) == 0 for s in listas):
        return 0
    
    docs_resultado = set.intersection(*listas)
    
    # Detectar qué columnas tiene el dataframe
    if 'review' in df.columns:
        text_col = 'review'
    else:
        text_col = 'review_content'
    
    # Solo mostrar si es necesario (para evitar spam en mediciones)
    return len(docs_resultado)

In [94]:
def medir_tiempo_busqueda(palabra):
    resultados = []
    
    # Búsqueda lineal (corpus pequeño)
    t0 = time.time()
    buscar(df['review'], palabra)
    t1 = time.time()
    
    # Búsqueda lineal (corpus grande)
    t2 = time.time()
    buscar(df_large['review_content'], palabra)
    t3 = time.time()
    
    # Búsqueda con índice invertido (corpus pequeño)
    t4 = time.time()
    buscar_rapido_flexible(palabra, indice_invertido, df)
    t5 = time.time()
    
    # Búsqueda con índice invertido (corpus grande)
    t6 = time.time()
    buscar_rapido_flexible(palabra, indice_invertido_large, df_large)
    t7 = time.time()
    
    resultados.append({
        'Palabra': palabra,
        'Lineal (50K)': round(t1 - t0, 4),
        'Lineal (1M)': round(t3 - t2, 4),
        'Índice (50K)': round(t5 - t4, 4),
        'Índice (1M)': round(t7 - t6, 4),
    })
    
    return pd.DataFrame(resultados)

In [95]:
palabras = ['family', 'love', 'boring', 'excellent', 'terrible']

resultados_df = pd.concat([medir_tiempo_busqueda(p) for p in palabras], ignore_index=True)

# Mostrar resultados
print("\n" + "="*70)
print("COMPARACIÓN DE TIEMPOS DE BÚSQUEDA (en segundos)")
print("="*70)
display(resultados_df)

# Calcular speedup (mejora)
resultados_df['Speedup (50K)'] = (
    resultados_df['Lineal (50K)'] / resultados_df['Índice (50K)']
).round(2)

resultados_df['Speedup (1M)'] = (
    resultados_df['Lineal (1M)'] / resultados_df['Índice (1M)']
).round(2)

print("\n" + "="*70)
print("FACTOR DE MEJORA (veces más rápido con índice)")
print("="*70)
display(resultados_df[['Palabra', 'Speedup (50K)', 'Speedup (1M)']])


COMPARACIÓN DE TIEMPOS DE BÚSQUEDA (en segundos)


,Palabra,Lineal (50K),Lineal (1M),Índice (50K),Índice (1M)
0,family,0.2312,0.9191,0.0004,0.0011
1,love,0.2988,0.9627,0.0010,0.0027
2,boring,0.2216,0.9232,0.0003,0.0003
3,excellent,0.3048,1.1073,0.0003,0.0004
4,terrible,0.2935,1.0919,0.0003,0.0002



FACTOR DE MEJORA (veces más rápido con índice)


,Palabra,Speedup (50K),Speedup (1M)
0,family,578.00,835.55
1,love,298.80,356.56
2,boring,738.67,3077.33
3,excellent,1016.00,2768.25
4,terrible,978.33,5459.50
